In [ ]:
import pandas as pd
import numpy as np

la_liga_teams = [
    ('Real Madrid','Real Madrid CF'), 
    ('Atlético Madrid','Atlético de Madrid'), 
    ('Real Sociedad','Real Sociedad'), 
    ('Sevilla','Sevilla FC'), 
    ('Betis','Real Betis Balompié'), 
    ('Rayo Vallecano','Rayo Vallecano'),
    ('Barcelona','FC Barcelona'),
    ('Athletic Club','Athletic Club de Bilbao'),
    ('Espanyol','RCD Espanyol de Barcelona'),
    ('Osasuna','CA Osasuna'),
    ('Valencia','Valencia CF'),
    ('Villarreal','Villarreal CF'),
    ('Celta Vigo','RC Celta de Vigo'),
    ('Mallorca','RCD Mallorca'),
    ('Alavés','Deportivo Alavés'),
    ('Granada','Granada CF'),
    ('Elche','Elche CF'),
    ('Cádiz','Cádiz CF'),
    ('Getafe','Getafe CF'),
    ('Levante','Levante Unión Deportiva'),
]

logos = [
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/53a2f082.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/db3b9613.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/e31d1cd9.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/ad2be733.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/fc536746.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/98e8af82.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/206d90db.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/2b390eca.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/a8661628.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/03c57e2b.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/dcc91a7b.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/2a8183b3.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/f25da7fb.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/2aa12281.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/8d6fd021.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/a0435291.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/6c8b07df.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/ee7c297c.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/7848bd64.png',
    'https://d2p3bygnnzw9w3.cloudfront.net/req/202201141/tlogo/fb/9800b6a1.png',
]

# Stat data processing

In [ ]:
# read the data (statistics)
df_yigit = pd.read_csv("stat_data.csv", header=0)

In [ ]:
# create a new header -- names for the columns

stats = ['min', '25', '50', '75', 'max']
features = ['goals', 'assists', 'gca', 'passes', 'aerials_won_pct', 'tackles']

header = ['squad', 'pos']

for feature in features:
    for stat in stats:
        c_name = feature + '_' + stat
        header.append(c_name)

In [ ]:
# drop the remaining header rows in the dataframe

tmp = df_yigit.drop([0,1]) # drop the header rows

In [ ]:
# create a dictionary for renaming the column names with something that makes sense

rename = {}
for old, new in zip(list(tmp.columns), header):
    rename[old] = new

tmp = tmp.rename(columns=rename)

In [ ]:
# remove GKs from the dataframe
df_yigit = tmp[(tmp.pos == 'DF') | (tmp.pos == 'MF') | (tmp.pos == 'FW')]

In [ ]:
df_yigit.head()

,squad,pos,goals_min,goals_25,goals_50,goals_75,goals_max,assists_min,assists_25,assists_50,assists_75,assists_max,gca_min,gca_25,gca_50,gca_75,gca_max,passes_min,passes_25,passes_50,passes_75,passes_max,aerials_won_pct_min,aerials_won_pct_25,aerials_won_pct_50,aerials_won_pct_75,aerials_won_pct_max,tackles_min,tackles_25,tackles_50,tackles_75,tackles_max
2,1. FC Köln,DF,0.0,0.0,0.0,1.0,7.0,0.0,0.0,0.5,2.75,7.0,0.0,0.5,4.0,7.25,13.0,405.0,1055.75,1421.5,1930.0,3540.0,65.5,92.67500000000001,142.45,165.95000000000002,201.0,19.0,34.5,62.5,124.0,141.0
3,1. FC Köln,FW,13.0,14.0,15.0,16.0,17.0,2.0,2.0,2.0,5.0,8.0,2.0,7.5,13.0,16.5,20.0,582.0,894.5,1207.0,1526.0,1845.0,140.9,150.10000000000002,159.3,189.7,220.1,15.0,23.0,31.0,60.5,90.0
5,1. FC Köln,MF,1.0,1.75,3.5,9.0,19.0,0.0,1.75,5.0,7.5,11.0,0.0,4.75,7.5,20.5,23.0,428.0,1067.75,1217.5,2120.5,5002.0,39.1,86.47500000000001,116.3,146.4,230.6,12.0,51.5,55.0,84.0,292.0
6,1. FC Union Berlin,DF,0.0,2.0,2.0,4.0,7.0,0.0,0.0,2.0,7.0,17.0,0.0,1.0,4.0,14.0,23.0,516.0,1461.0,3502.0,3591.0,5844.0,120.9,123.0,179.0,214.1,235.2,23.0,69.0,131.0,147.0,160.0
7,1. FC Union Berlin,FW,3.0,4.75,7.5,13.25,25.0,1.0,2.0,2.5,6.0,17.0,5.0,6.0,7.0,12.5,34.0,367.0,495.0,664.5,902.25,2879.0,55.3,76.55,89.30000000000001,91.175,110.5,11.0,16.5,18.0,24.75,57.0


# FBref processing

In [ ]:
df = pd.read_csv("top5_with_photos.csv")

In [ ]:
df = df[list(df.columns)[1:]]

In [ ]:
df = df[df.season == 2022]

In [ ]:
df.head()

,season,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,assists,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,minutes_90s,shots_total,shots_on_target,shots_free_kicks,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,npxg_per_shot,xg_net,...,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,games_subs,minutes_per_sub,unused_subs,points_per_match,on_goals_for,on_goals_against,plus_minus,plus_minus_per90,plus_minus_wowy,on_xg_for,on_xg_against,xg_plus_minus,xg_plus_minus_per90,xg_plus_minus_wowy,wc_starts,wc_games,wc_minutes,wc_goals,wc_assists,wc_cards_yellow,wc_cards_red,overall,potential,pace,shooting,passing,dribbling,defending,physic,wage_eur,value_eur,photo
0,2022,Aaron Connolly,ie IRL,FW,Brighton & Hove Albion,21,2000,45.0,24.0,2205.0,5.0,2.0,0.0,0.0,0.0,0.0,0.44,0.18,0.63,0.44,0.63,7.2,7.2,0.9,0.89,0.24,1.14,0.89,1.14,24.5,63.0,21.0,0.0,69.0,6.49,1.84,0.17,0.48,0.46,-2.2,...,22.0,3.0,0.0,0.0,89.0,25.0,85.0,49.4,6.0,14.0,27.0,1.78,23.0,13.0,10.0,0.74,0.46,20.4,14.1,6.3,0.47,0.22,0,0,0,0,0,0,0,70.0,77.0,75.0,70.0,50.0,71.0,20.0,64.0,23000.0,3300000.0,https://fbref.com/req/202005121/images/headsho...
1,2022,Aaron Cresswell,eng ENG,DF,West Ham United,32,1989,100.0,98.0,8626.0,4.0,10.0,0.0,0.0,11.0,0.0,0.18,0.37,0.55,0.18,0.55,3.1,3.1,10.0,0.13,0.39,0.51,0.13,0.51,95.9,57.0,11.0,25.0,71.1,2.46,0.44,0.28,1.10,0.21,0.9,...,4.0,0.0,0.0,0.0,883.0,130.0,88.0,242.5,23.0,42.0,27.0,4.08,58.0,70.0,-12.0,-1.58,-0.73,58.6,69.1,-10.7,-1.24,-0.87,0,0,0,0,0,0,0,79.0,79.0,70.0,60.0,75.0,76.0,77.0,68.0,66000.0,12500000.0,https://fbref.com/req/202005121/images/headsho...
2,2022,Aaron Hickey,sct SCO,DF,Bologna,19,2002,29.0,27.0,2121.0,3.0,0.0,0.0,0.0,7.0,1.0,0.20,0.00,0.20,0.20,0.20,0.6,0.6,0.6,0.04,0.06,0.10,0.04,0.10,23.5,16.0,8.0,1.0,87.1,1.22,0.58,0.23,0.43,0.07,2.4,...,0.0,0.0,0.0,0.0,173.0,12.0,22.0,76.2,6.0,67.0,9.0,3.61,43.0,31.0,12.0,1.08,-1.02,43.0,30.4,12.7,1.10,-0.24,0,0,0,0,0,0,0,69.0,82.0,69.0,50.0,65.0,67.0,66.0,56.0,7000.0,3200000.0,https://fbref.com/req/202005121/images/headsho...
3,2022,Aaron Lennon,eng ENG,MF,Burnley,34,1987,38.0,19.0,1826.0,1.0,1.0,0.0,0.0,6.0,0.0,0.07,0.07,0.15,0.07,0.15,0.9,0.9,0.7,0.17,0.06,0.24,0.17,0.24,20.3,7.0,2.0,0.0,50.0,2.05,0.15,0.25,0.50,0.28,0.1,...,0.0,0.0,1.0,0.0,118.0,4.0,21.0,36.4,13.0,63.0,15.0,2.77,51.0,69.0,-18.0,-2.26,-0.72,47.0,63.7,-16.6,-1.76,-0.72,0,0,0,0,0,0,0,73.0,73.0,78.0,60.0,70.0,77.0,48.0,56.0,28000.0,1400000.0,https://fbref.com/req/202005121/images/headsho...
6,2022,Aaron Ramsdale,eng ENG,GK,Arsenal,23,1998,90.0,90.0,8100.0,0.0,1.0,0.0,0.0,2.0,0.0,0.00,0.03,0.03,0.00,0.03,0.0,0.0,0.3,0.00,0.00,0.00,0.00,0.00,90.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,1.0,0.0,461.0,0.0,0.0,0.0,22.0,36.0,18.0,2.17,17.0,21.0,-4.0,-0.58,-0.13,16.9,22.9,-6.0,-0.82,0.01,0,0,0,0,0,0,0,74.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,36000.0,5000000.0,https://fbref.com/req/202005121/images/headsho...


In [ ]:
import json

file_name = 'laliga.json'

# makes JSON file for our MongoDB database
def team_dictionary(df_team, df_team_stats, team_name, logo):

    df_team = df_team.where(pd.notnull(df_team), None)
    df_team = df_team.replace({np.nan: None}) # <---
    players = df_team.T.to_dict()
    team = [players[key] for key in players]

    s = df_team_stats.T.to_dict()
    statistics = [s[key] for key in s]

    final_object = {
        'team': team_name,
        'logo': logo,
        'stats': statistics,
        'players': team
    }

    return final_object


league_dictionary = {
    'league_name': 'Spain Primera Division',
    'teams': [],
    }

counter = 0

for a, logo in zip(la_liga_teams, logos):

    _, team_name = a

    print(team_name)
    league_dictionary['teams'].append(
        team_dictionary(
            df[df.squad == team_name], 
            df_yigit[df_yigit.squad == team_name].drop('squad', axis=1).reset_index(drop=True),
            team_name,
            logo))
    
    counter += 1

with open(file_name, "w") as outfile:
    json.dump(league_dictionary, outfile)

Real Madrid CF
Atlético de Madrid
Real Sociedad
Sevilla FC
Real Betis Balompié
Rayo Vallecano
FC Barcelona
Athletic Club de Bilbao
RCD Espanyol de Barcelona
CA Osasuna
Valencia CF
Villarreal CF
RC Celta de Vigo
RCD Mallorca
Deportivo Alavés
Granada CF
Elche CF
Cádiz CF
Getafe CF
Levante Unión Deportiva
